### SPLIT AFRICA DATASET 128

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  

RESULTS:

```
-------------------------------------------------------------------------------------
block relative split: 0.7015503875968992 0.1996124031007752 0.09883720930232558 0.19875776397515527
block split fracs: 0.562111801242236 0.15993788819875776 0.07919254658385093 0.19875776397515527
-------------------------------------------------------------------------------------

train group frac: 0.5941516555974963
* low-med-high rfracs: 1.0020939411588003 0.9951734321859522 1.0218651272330403

valid group frac: 0.14080940322041186
* low-med-high rfracs: 0.9921465495165993 1.0299155228798504 0.8543516919206738

test group frac: 0.07018968894834918
* low-med-high rfracs: 1.0154040220265386 1.0182645611266203 0.964089938938565

hold_out group frac: 0.19484925223374266
* low-med-high rfracs: 0.9937414055683951 0.9865195401044039 1.0515165734345673

```

NOTES:

    1. validation is a little low in group count but that speeds up training. its the test group that matters
    2. the low-med-high fracs are good. 
    3. the hold out group can be used for additional training or additional validation if needed.
 

---

In [1]:
import pandas as pd
from random import sample

In [2]:
WSIZE=128
DSET=f'surface-water.africa.win{WSIZE}'
NO_DATA_MAX=0.05
IDENTS=['group_id','window_index']

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df.sample(3)

24762 684 24338
21278 644 20929


,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset
19045,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_29_-29,group_29.53_-29.74,9,"(256, 128, 128, 128)",0.267944,0.730652,0.001404,africa_mtn
21744,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_25_-30,group_25.55_-30.55,14,"(384, 256, 128, 128)",0.571350,0.427917,0.000732,africa_mtn
3263,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-5_32,group_-5.84_32.75,7,"(128, 384, 128, 128)",0.232727,0.760010,0.007263,africa


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=IDENTS).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group frac:',frac)
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=IDENTS).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

-------------------------------------------------------------------------------------
block relative split: 0.7015503875968992 0.1996124031007752 0.09883720930232558 0.19875776397515527
block split fracs: 0.562111801242236 0.15993788819875776 0.07919254658385093 0.19875776397515527
-------------------------------------------------------------------------------------

train group frac: 0.5941516555974963
* low-med-high rfracs: 1.0020939411588003 0.9951734321859522 1.0218651272330403

valid group frac: 0.14080940322041186
* low-med-high rfracs: 0.9921465495165993 1.0299155228798504 0.8543516919206738

test group frac: 0.07018968894834918
* low-med-high rfracs: 1.0154040220265386 1.0182645611266203 0.964089938938565

hold_out group frac: 0.19484925223374266
* low-med-high rfracs: 0.9937414055683951 0.9865195401044039 1.0515165734345673


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset,water_tag,data_split
15802,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_10_8,group_10.12_8.49,4,"(128, 0, 128, 128)",0.017700,0.978638,0.003662,africa_mtn,medium,train
6939,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_31_27,group_31.16_27.22,5,"(128, 128, 128, 128)",0.037720,0.962280,0.000000,africa,medium,train
24040,gs://surface-water-public/data/v1/jrc/africa_s...,gs://surface-water-public/data/v1/sentinel_1/a...,block_32_26,group_32.24_26.06,11,"(256, 384, 128, 128)",0.169495,0.830505,0.000000,africa_small,medium,hold_out


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.win128.split.csv gs://surface-water-public/data/v1/datasets


---

In [15]:
# !gsutil cp surface-water.africa.win128.split.csv gs://surface-water-public/data/v1/datasets

---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

6.78027873317265

In [13]:
_df.drop_duplicates(subset=IDENTS)[['water','not_water']].describe()

,water,not_water
count,15382.000000,15382.000000
mean,0.127388,0.867780
std,0.141102,0.142047
min,0.005005,0.370117
25%,0.025162,0.814941
50%,0.069946,0.924805
75%,0.178024,0.970093
max,0.599976,0.994995
